In [1]:
print("Ok")

Ok


In [2]:
%pwd

'e:\\Bappy\\Coding\\Youtube\\Generative AI\\Langchain-Projects\\End-to-end-Medical-Chatbot-Generative-AI\\research'

In [4]:
import os
os.chdir("../")

In [5]:
%pwd

'/home/yasir/Downloads/backup_aug_24/Data/Projects/End-to-end-Chatbot'

In [1]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [9]:
extracted_data=load_pdf_file(data='Data/')

In [10]:
extracted_data

[Document(metadata={'source': 'Data/pakistan_constitution.pdf', 'page': 0}, page_content='THE \nCO N STITU TIO N  \nO F THE \nISLA M IC R EPU B LIC \nO F PA K ISTA N \n[A s m odified upto the 31 st M ay, 2018] \nN ATIO N A L A SSEM BLY  O F PA K ISTA N \n'),
 Document(metadata={'source': 'Data/pakistan_constitution.pdf', 'page': 1}, page_content=''),
 Document(metadata={'source': 'Data/pakistan_constitution.pdf', 'page': 2}, page_content='PR EFA C E \nThe N ational A ssem bly of Pakistan passed the Constitution on \n10th A pril, 1973, the President of the A ssem bly authenticated it on 12 th \nA pril, 1973 and the A ssem bly published the Constitution of the \nIslam ic R epublic of Pakistan. Since then, a num ber of am endm ents have \nbeen m ade therein and it has becom e necessary and expedient that an \nup-to-date and authentic version of the Constitution be published by the \nA ssem bly. \nThis Eighth Edition, w hich is intended to provide an updated \nversion of the Constitution, 

In [11]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [12]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 1084


In [13]:
text_chunks

[Document(metadata={'source': 'Data/pakistan_constitution.pdf', 'page': 0}, page_content='THE \nCO N STITU TIO N  \nO F THE \nISLA M IC R EPU B LIC \nO F PA K ISTA N \n[A s m odified upto the 31 st M ay, 2018] \nN ATIO N A L A SSEM BLY  O F PA K ISTA N'),
 Document(metadata={'source': 'Data/pakistan_constitution.pdf', 'page': 2}, page_content='PR EFA C E \nThe N ational A ssem bly of Pakistan passed the Constitution on \n10th A pril, 1973, the President of the A ssem bly authenticated it on 12 th \nA pril, 1973 and the A ssem bly published the Constitution of the \nIslam ic R epublic of Pakistan. Since then, a num ber of am endm ents have \nbeen m ade therein and it has becom e necessary and expedient that an \nup-to-date and authentic version of the Constitution be published by the \nA ssem bly.'),
 Document(metadata={'source': 'Data/pakistan_constitution.pdf', 'page': 2}, page_content='A ssem bly. \nThis Eighth Edition, w hich is intended to provide an updated \nversion of the Consti

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [27]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [28]:
embeddings = download_hugging_face_embeddings()

In [29]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [30]:
query_result

[-0.034477315843105316,
 0.031023172661662102,
 0.006734910886734724,
 0.02610892429947853,
 -0.03936195746064186,
 -0.1603025197982788,
 0.06692396104335785,
 -0.006441440898925066,
 -0.04745054617524147,
 0.014758836477994919,
 0.07087532430887222,
 0.055527545511722565,
 0.01919332519173622,
 -0.026251299306750298,
 -0.01010951679199934,
 -0.026940451934933662,
 0.022307397797703743,
 -0.022226639091968536,
 -0.1496926248073578,
 -0.01749303936958313,
 0.007676327601075172,
 0.054352276027202606,
 0.0032544792629778385,
 0.03172592446208,
 -0.08462144434452057,
 -0.029405953362584114,
 0.05159562826156616,
 0.048124104738235474,
 -0.003314818488433957,
 -0.05827919766306877,
 0.04196928068995476,
 0.02221069671213627,
 0.12818878889083862,
 -0.02233896404504776,
 -0.011656257323920727,
 0.06292840093374252,
 -0.03287629410624504,
 -0.09122602641582489,
 -0.031175386160612106,
 0.05269954726099968,
 0.047034841030836105,
 -0.08420310169458389,
 -0.030056146904826164,
 -0.020744822919

In [31]:
from dotenv import load_dotenv
load_dotenv()

True

In [32]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GROQ_API_KEY=os.environ.get('GROQ_API_KEY')

In [34]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "pak-law"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [35]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [36]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [37]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [38]:
docsearch

In [39]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [40]:
retrieved_docs = retriever.invoke("What are the fundamental rigths of the citizens?")

In [41]:
retrieved_docs

[Document(metadata={'page': 38.0, 'source': 'Data/pakistan_constitution.pdf'}, page_content='38 . The State shall–– \n(a) secure the w ell-being of the people, irrespective of \nsex, caste, creed or race, by raising their standard of \nliving, by preventing the concentration of w ealth and \nm eans of production and distribution in the hands of a \nfew  to the detrim ent of general interest and by \nensuring equitable adjustm ent of rights betw een \nem ployers and em ployees, and landlords and tenants; \n(b) provide for all citizens, w ithin the available resources'),
 Document(metadata={'page': 34.0, 'source': 'Data/pakistan_constitution.pdf'}, page_content='services such as roads, w ater supply, \nsew erage, gas and electric pow er to all or any \nspecified class of citizens; or \n(iii) providing m aintenance to those w ho, on account \nof unem ploym ent, sickness, infirm ity or old age, \nare unable to m aintain them selves; or \n(f) any existing law  or any law  m ade in pursuance

In [42]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature=0.4,
    max_tokens=500,
    groq_api_key=os.environ.get('GROQ_API_KEY')
)

In [43]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [44]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [45]:
response = rag_chain.invoke({"input": "What are the fundamental rigths of the citizens?"})
print(response["answer"])

The fundamental rights of the citizens, as outlined in the provided context, include the right to equality, freedom of speech, expression, faith, worship, and association, subject to law and public morality. The State is responsible for securing the well-being of the people, providing services such as roads, water supply, sewage, gas, and electric power, and maintaining those who are unable to maintain themselves due to unemployment, sickness, infirmity, or old age. Additionally, adequate compensation, protection of minorities and backward classes, and the independence of the judiciary are ensured, along with the safeguarding of the Federation's territorial integrity, independence, and sovereign rights.


In [46]:
response = rag_chain.invoke({"input": "What is status of Gilgit Baltistan?"})
print(response["answer"])

Based on the provided context, the Constitution of Pakistan does not specifically mention the status of Gilgit Baltistan. The text includes amendments that substitute terms like "Baluchistan", "North-West Frontier", and "Sind", but there is no direct reference to Gilgit Baltistan. Therefore, I cannot provide an accurate answer to its status based on this information.
